<a href="https://colab.research.google.com/github/YapingWu/GoogleColab/blob/main/genetate_password.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# github代码复制到google colab

## 配置ssh

In [ ]:
!rm -rf ~/.ssh
!unzip '/content/ssh-colab.zip' -d ~/.ssh
!chmod 700 ~/.ssh

# add the ssh server as a hnown host
!touch ~/.ssh/known_hosts
!ssh-keyscan github.com >> ~/.ssh/known_hosts
!chmod 644 ~/.ssh/known_hosts

In [ ]:
!ssh-agent /bin/bash 

# 以下命令在ssh-agent启动的shell中执行
# chmod 600 ~/.ssh/id_rsa_colab  # 私钥需要设置仅自己可以访问，才能添加到代理
# ssh-add ~/.ssh/id_rsa_colab
# ssh-add -l
# exit

## 挂载google云盘

In [ ]:
from google.colab import drive
drive.mount('./drive')

import os
os.chdir('/content/drive/MyDrive/')

!pwd
!ls

In [7]:
!git config --global user.name 'colab'
!git config --global user.email 'vyapings@163.com'

## 更新代码

In [15]:
# !git clone git@github.com:YapingWu/generate_password.git

import os
os.chdir('/content/drive/MyDrive/generate_password')
!pwd
!ls

!git stash
!git stash clear
!git pull origin main

/content/drive/MyDrive/generate_password
conf  dataset  logs  main.py  README.md  requirements.txt  result  src
No local changes to save
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), done.
From github.com:YapingWu/generate_password
 * branch            main       -> FETCH_HEAD
   eabf653..c1b208b  main       -> origin/main
Updating eabf653..c1b208b
Fast-forward
 main.py             |  3 +-
 src/char_lstm.py    | 83 ++++++++++++++++++++++++++++-------------------------
 src/markov_model.py |  4 ++-
 3 files changed, 49 insertions(+), 41 deletions(-)


# 运行项目

In [ ]:
# 修改工作目录
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/generate_password')
print(os.getcwd())

# 修改时区
!date -R
import os
os.environ['TZ'] = "Asia/Shanghai"
!date -R

## 分析数据

In [9]:
!python main.py -e pro --log-level info \
  --stats-only \
  --pwd-file dataset/en/myspace.txt dataset/en/phpbb.txt

2021-04-16 08:48:51.013918: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-16 08:48:53,161 - main.py[:71] - INFO - Beginning...
2021-04-16 08:48:53,515 - main.py[:72] - INFO - args：{'proj_env': 'pro', 'log_level': 'info', 'pwd_file': ['dataset/en/myspace.txt', 'dataset/en/phpbb.txt'], 'stats_only': True, 'train': False, 'generated': False, 'model': 'char_lstm', 'model_file': None, 'evaluate': False, 'data_name': None, 'test_file': None}
2021-04-16 08:48:53,515 - main.py[:75] - INFO - 分析密码分布...
2021-04-16 08:48:54,375 - common.py[:38] - INFO - dataset/en/myspace.txt数据集中的密码数量：37144
2021-04-16 08:48:54,423 - common.py[:47] - INFO - 密码长度不在[4, 20]区间的密码个数：259，占总密码数量的百分比：0.70%
2021-04-16 08:48:55,249 - common.py[:38] - INFO - dataset/en/phpbb.txt数据集中的密码数量：184379
2021-04-16 08:48:55,342 - common.py[:47] - INFO - 密码长度不在[4, 20]区间的密码个数：1013，占总密码数量的百分比：0.55%
2021-04-16 08:48:55,345 - main.py[:126] - INFO - End...


## Char Lstm

### 训练模型

#### myspace

In [ ]:
!python main.py -e pro --log-level info \
  --train --model char_lstm \
  --pwd-file dataset/en/myspace.txt

#### phpbb

In [ ]:
!python main.py -e pro --log-level info \
  --train --model char_lstm \
  --pwd-file dataset/en/phpbb.txt

#### tensorboard

In [10]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir 'logs/tb_logs/char_lstm/20210414193152'

In [ ]:
%tensorboard --logdir 'logs/tb_logs/char_lstm/20210415175500'

### 生成密码

#### myspace

In [ ]:
!python main.py -e pro --log-level info \
  --generated --model char_lstm \
  --data-name myspace

#### phpbb

In [12]:
!python main.py -e pro --log-level info \
  --generated --model char_lstm \
  --data-name phpbb

2021-04-16 08:55:16.138387: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-16 08:55:17,985 - main.py[:71] - INFO - Beginning...
2021-04-16 08:55:17,986 - main.py[:72] - INFO - args：{'proj_env': 'pro', 'log_level': 'info', 'pwd_file': None, 'stats_only': False, 'train': False, 'generated': True, 'model': 'char_lstm', 'model_file': None, 'evaluate': False, 'data_name': ['phpbb'], 'test_file': None}
2021-04-16 08:55:18,258 - char_lstm.py[:221] - INFO - [*] Generate password...
2021-04-16 08:55:18,259 - char_lstm.py[:216] - INFO - model file：result/model/char_lstm/phpbb.h5
2021-04-16 08:55:18.696796: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-04-16 08:55:18.719943: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-04-16 08:55:18.784799: E tensorflow/stream_executor/cuda/cuda_d

### 评估模型

In [13]:
!python main.py -e pro --log-level info \
  --evaluate --model char_lstm \
  --data-name myspace phpbb \
  --test-file dataset/en/myspace.txt dataset/en/phpbb.txt dataset/en/rockyou_small.txt

2021-04-16 10:34:56.358037: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-16 10:34:58,336 - main.py[:71] - INFO - Beginning...
2021-04-16 10:34:58,542 - main.py[:72] - INFO - args：{'proj_env': 'pro', 'log_level': 'info', 'pwd_file': None, 'stats_only': False, 'train': False, 'generated': False, 'model': 'char_lstm', 'model_file': None, 'evaluate': True, 'data_name': ['myspace', 'phpbb'], 'test_file': ['dataset/en/myspace.txt', 'dataset/en/phpbb.txt', 'dataset/en/rockyou_small.txt']}
2021-04-16 10:34:58,544 - char_lstm.py[:242] - INFO - [*] Evaluate model(myspace)...
2021-04-16 10:34:58,544 - char_lstm.py[:244] - INFO - 生成的密码文件：result/guessed/char_lstm/myspace.*.txt
2021-04-16 10:34:58,604 - char_lstm.py[:252] - INFO - 测试集文件：dataset/en/myspace.txt，测试集大小：5000
2021-04-16 10:34:58,858 - char_lstm.py[:260] - INFO - alpha：4, generated：10086, guessed: 6, mr：0.12%
2021-04-16 10:35:01,240 - char_lstm.py[:260] - INFO

## Markov模型

### 训练模型

In [ ]:
!python main.py -e pro --log-level info \
  --train --model markov \
  --pwd-file dataset/en/myspace.txt dataset/en/phpbb.txt

### 生成密码

#### myspace

In [ ]:
!python main.py -e pro --log-level info \
  --generated --model markov \
  --data-name myspace

#### phpbb

In [ ]:
!python main.py -e pro --log-level info \
  --generated --model markov \
  --data-name phpbb

### 评估模型

In [14]:
!python main.py -e pro --log-level info \
  --evaluate --model markov \
  --data-name myspace phpbb \
  --test-file dataset/en/myspace.txt dataset/en/phpbb.txt dataset/en/rockyou_small.txt

2021-04-16 10:48:08.032027: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-04-16 10:48:09,797 - main.py[:71] - INFO - Beginning...
2021-04-16 10:48:10,002 - main.py[:72] - INFO - args：{'proj_env': 'pro', 'log_level': 'info', 'pwd_file': None, 'stats_only': False, 'train': False, 'generated': False, 'model': 'markov', 'model_file': None, 'evaluate': True, 'data_name': ['myspace', 'phpbb'], 'test_file': ['dataset/en/myspace.txt', 'dataset/en/phpbb.txt', 'dataset/en/rockyou_small.txt']}
2021-04-16 10:48:10,003 - markov_model.py[:151] - INFO - [*] Evaluate model(myspace)...
2021-04-16 10:48:10,003 - markov_model.py[:153] - INFO - 生成的密码文件：result/guessed/markov/myspace.*.txt
2021-04-16 10:48:11,390 - markov_model.py[:161] - INFO - 测试集文件：dataset/en/myspace.txt，测试集大小：5000
2021-04-16 10:48:11,778 - markov_model.py[:169] - INFO - alpha：4, generated：9953, guessed: 1, mr：0.02%
2021-04-16 10:48:15,064 - markov_model.py[:169

## 画图

In [ ]:
import matplotlib.pyplot as plt